In [7]:
from langchain.llms.sagemaker_endpoint import LLMContentHandler
import json
from typing import Dict

class ContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs: Dict) -> bytes:
        input_str = json.dumps({"inputs": prompt, **model_kwargs})
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> str:
        response_json = json.loads(output.read().decode("utf-8"))
        return response_json[0]["generated_text"]

In [9]:
from langchain import LLMChain
from langchain.prompts import PromptTemplate
from langchain import SagemakerEndpoint

prompt_template="""User may ask some cryptocurrency's market trend, or ask some investion advise.
Please generate a complete question, using user's input below.
User's input:{user_input}
Complete question:"""
prompt=PromptTemplate.from_template(prompt_template)
content_handler = ContentHandler()
llm=SagemakerEndpoint(
                endpoint_name="huggingface-pytorch-tgi-inference-2023-07-24-07-23-15-934", 
                # credentials_profile_name="default", 
                region_name="us-east-1", 
                model_kwargs={
                    "parameters":{
                        "do_sample":True,
                        # "top_p": 0.9,
                        # "top_k": 10,
                        "repetition_penalty": 1.03,
                        "max_new_tokens":1024,
                        "temperature":0.1,
        				"return_full_text":False,
                        # "max_length":2048,
                        "truncate":2048,
                        # "num_return_sequences":2000,
                        # "stop": ["\n"],
                        },
                    },
        		content_handler=content_handler,
            verbose=True,
)
chain=LLMChain(llm=llm,prompt=prompt,verbose=True)
user_inputs=input("Your prompt: ")
print(f"{chain(user_inputs)}")

Your prompt:  swftc quote




> Entering new  chain...
Prompt after formatting:
User may ask some cryptocurrency's market trend, or ask some investion advise.
Please generate a complete question, using user's input below.
User's input:swftc quote
Complete question:

> Finished chain.
{'user_input': 'swftc quote', 'text': 'What is the current market trend for SWFTC (SwiftCoin) and what are some investment advice for buying/selling this cryptocurrency?'}


In [ ]:
sss